In [1]:
import keras
keras.__version__

Using TensorFlow backend.


'2.2.4'

# Text generation with a LSTM

We are going to implement a LSTM in Keras. The first thing we need is a big amount of text to be able to learn a linguistic model. One can use any big text file. In this example we are going to be using El Quijote. Our model will learn a specific model based on the writting style of Cervantes in this particular book.


## Preparing the data

First we are going to dowload the corpus and convert it to lower case letters.

In [2]:
import keras
import numpy as np

path = keras.utils.get_file(
    'quijote.txt',
    origin='https://gist.githubusercontent.com/jsdario/6d6c69398cb0c73111e49f1218960f79/raw/8d4fc4548d437e2a7203a5aeeace5477f598827d/el_quijote.txt')
text = open(path).read().lower()
print('Longitud del corpus:', len(text))

1064960/1060259 [==============================] - 0s 0us/step
Longitud del corpus: 1038397


Next we will extract sentences with a partial overlapping of lenght `maxlon`, we will transform them into a one-hot vector and we will then store it in a 3D numpy array `x` whose structure will correspond to `n_sentences, maxlon, unique_characters`.
Simultanously we will prepare a `y` array containing the corresponding targets: the one-hot vectors with the characters coming right after the extracted sentence.

In [3]:
# Length of extracted character sequences
maxlon = 60

# We sample a new sequence every `step` characters
step = 3

# This holds our extracted sequences
sentences = []

# This holds the targets (the follow-up characters)
next_chars = []

for i in range(0, len(text) - maxlon, step):
    sentences.append(text[i: i + maxlon])
    next_chars.append(text[i + maxlon])
print('Number of sentences:', len(sentences))

# List of unique characters in the corpus
chars = sorted(list(set(text)))
print('Unique characters:', len(chars))
# Dictionary mapping unique characters to their index in `chars`
char_indices = dict((char, chars.index(char)) for char in chars)

# Next, one-hot encode the characters into binary arrays.
print('Vectorization...')
x = np.zeros((len(sentences), maxlon, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

Number of sentences: 346113
Unique characters: 65
Vectorization...


## Building the network

Our net is just one single `LSTM`followed by a `dense` classifier and a softmax for all the possible characters. 


In [18]:
from keras import layers
from keras.layers import LSTM, Dense, Activation

model = keras.models.Sequential()
model.add(LSTM(3))
model.add(Dense(65))
model.add(Activation('softmax'))

Since our targets are one-hot vectors, we will use `categorical_crossentropy` as loss function of our model. Use RMP prop as optimizer.

In [19]:
model.compile(optimizer = "rmsprop", loss = "categorical_crossentropy")

## Training the model and sampling from it


Given a trained model and a text fragment as seed, we can generate a new text following these steps:

*  Extract from the model the probability distribution of the given text given till that particular moment
* Reweights the distribution for a certain "temperature"
* Randomly sample the following character randomly following the reweighted distribution
* Add the character at the end of the text

With this code we reweights the original probability coming from the model and extract an index (sampling function)



In [20]:
def sample(preds, temperatura=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperatura
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

Finally, we have here the loop inside of which we will do the training and generate the text

In [21]:
import random
import sys

for epoch in range(1, 20):
    print('Epoch: ', epoch)
    # Fit the model for 1 epoch on the available training data
    model.fit(x, y,
              batch_size=128,
              epochs=1)

    # Select a text seed at random
    start_index = random.randint(0, len(text) - maxlon - 1)
    generated_text = text[start_index: start_index + maxlon]
    print('--- Generating with the following seed: "' + generated_text + '"')

    for temperatura in [0.3]:
        print('------ Temperature:', temperatura)
        sys.stdout.write(generated_text)

        # We generate 400 characters
        for i in range(400):
            sampled = np.zeros((1, maxlon, len(chars)))
            for t, char in enumerate(generated_text):
                sampled[0, t, char_indices[char]] = 1.

            preds = model.predict(sampled, verbose=0)[0]
            next_index = sample(preds, temperatura)
            next_char = chars[next_index]

            generated_text += next_char
            generated_text = generated_text[1:]

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()

Epoch:  1
Epoch 1/1
346113/346113 [==============================] - 414s 1ms/step - loss: 2.9521
--- Generating with the following seed: "ir -dijo camila- que lo que cuesta poco se estima en menos.
"
------ Temperature: 0.3
ir -dijo camila- que lo que cuesta poco se estima en menos.
o  a ee a e eo e e na e au de ue ua ie es e a eo aa de ieo e e ia ea ere ales uo o o io o e ea uo mio a le e ue a a ee e ue ace aso a eo e ns ua ace coa rue e vate al ua e iaso  e e e ele  ao  eee a ua a haa  eee e o e ue ue te a oda e aa a ae ia le re e o  é a e de nne uo o e ala ie o   oe eaua o na ara aar e e e sase a o mao ena aa  o a ae ua a aa o ia o e co eiue o e o ta aqa o ue o aa an ano 
Epoch:  2
Epoch 1/1
346113/346113 [==============================] - 429s 1ms/step - loss: 2.5063
--- Generating with the following seed: "jor informado, digo que los trueques, si es que tienes dello"
------ Temperature: 0.3
jor informado, digo que los trueques, si es que tienes dello a qela da a so an ase e de a

l loco, que a cada paso le alborotaba la venta. finalmente, cor al die lis en has de de de a la ha se pan de le as se no con que de penta a a de pente de an de pen en de de la la a se de de an pan para lo de y an de que de le de de con de pen di de le que me do tar lo col casa dento pen er mente diro pen de an mas que de de en an as al per cana de y as sunde tente caso le que mes pan la pen de de de ho as an as el aba de an le an da con de de se le que me 
Epoch:  13
Epoch 1/1
346113/346113 [==============================] - 498s 1ms/step - loss: 2.1697
--- Generating with the following seed: "acedaba a entrar en su casa, ni podía moverme a parte algun"
------ Temperature: 0.3
acedaba a entrar en su casa, ni podía moverme a parte algunte dente de ho y as de y mar as de de de an donte que me de me de de an cul que se pen lo su de mo se de que que lo de que la en le an as de de cana me la cana par an de me a ena can sur as con se la vente mente de poma de per que de de sur en de que se


## Tasks

* Use your own corpus instead of El Quijote (can be in another language)
* Modify the loop in order to take several different temperatures (between 0.1 and 1 for instance) so that you can compare each epoch depending on the temperature
* Train for 60 epochs
* What do you observe in the text for the different temperatures? Which seems to be the "best" temperature and why?













For this exercise, I will use the movie corpus found in https://github.com/dariusk/harpooneers/blob/master/corpus.txt.

In [32]:
path = keras.utils.get_file(
    'corpus.txt',
    origin='https://raw.githubusercontent.com/dariusk/harpooneers/master/corpus.txt')
text = open(path).read().lower()
print('Longitud del corpus:', len(text))

18317312/18314868 [==============================] - 1s 0us/step
Longitud del corpus: 18308757


In [34]:
import random
import sys

for epoch in range(1, 60):
    print('Epoch: ', epoch)
    # Fit the model for 1 epoch on the available training data
    model.fit(x, y,
              batch_size=128,
              epochs=1)

    # Select a text seed at random
    start_index = random.randint(0, len(text) - maxlon - 1)
    generated_text = text[start_index: start_index + maxlon].replace('@', '').replace('_', '').replace('|', '')
    print('--- Generating with the following seed: "' + generated_text + '"')

    for temperatura in [0.1, 0.25, 0.5, 0.75, 1.0]:
        print('------ Temperature:', temperatura)
        sys.stdout.write(generated_text)

        # We generate 400 characters
        for i in range(400):
            sampled = np.zeros((1, maxlon, len(chars)))
            for t, char in enumerate(generated_text):
                sampled[0, t, char_indices[char]] = 1.

            preds = model.predict(sampled, verbose=0)[0]
            next_index = sample(preds, temperatura)
            next_char = chars[next_index]

            generated_text += next_char
            generated_text = generated_text[1:]

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()

Epoch:  1
Epoch 1/1
346113/346113 [==============================] - 414s 1ms/step - loss: 2.1143
--- Generating with the following seed: "ee, as that thou dost not turn unto the beautiful garden, bl"
------ Temperature: 0.1
ee, as that thou dost not turn unto the beautiful garden, blo de de de de de de que de de de de de de de de de a la de de de de la de de de de de de de de de me de de de la de de de cala de que de de de de de as de a de de de de con de de la de de de de de de de de de de la de la de de de de de de la de de de de de de la de de de de as canta de de de de de de de de de de de de as de de de de la de de de pante de de de de de de de de as de de de de que de d
------ Temperature: 0.25
 de de pante de de de de de de de de as de de de de que de de ante anta se cora de que lo de la se de de de me se cona y de con lo la de que gara la y en con cado en lo de se le que a de la de a de de de as le a de de la de de cos de de se la la en y lo cana anta de y as que de se pasa de

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:3: RuntimeWarning: divide by zero encountered in log
  This is separate from the ipykernel package so we can avoid doing imports until


 de te de as con de cantes el le lo que man y muscuestila de lo dico con cueremo a se pen de de core me que mer se la cara siles lecés calos que se tonte su de y de vol ildo a que daste de o hen la corer cosa y y cara colen se senco que dino as de y de le cala de lo pontade ir pon cocuri quido que a de do as de adia heste pen la de las de du de de la me a de que se mer esta te
------ Temperature: 0.75
este pen la de las de du de de la me a de que se mer esta tendo uĺento toso o anto gentrebur caro csulel de aran lo coĺa que cadorré rusatisteridanerpanocancia su espatenacos pieyo, fua ma, con la come és que loncos que muriesas los te parren li quento y borepumos se que na cón sentis conde pante teendo
na y lerino lo, lan se di toles ás man balmo que me dico dasta meestesmédca si en pasti rico la que de án, le lo la, hosziebo que he lo paa de mont
------ Temperature: 1.0
rico la que de án, le lo la, hosziebo que he lo paa de montar corceson asa. en suve y poca y pido en conpobu

e cala de con a de a y de de a le pante la de a de de a que sedon mueyo y de mel de que col de dol a se de lós menta de le pen pos pal as dos as de mán que pura modo de me las cos -y de de se la y lando, y canta canas sar caa casa en y lo en en y as le adere en as palte, y de ando su de ne caltarta de la pasdia la la dé hasas pen de vela de a mes de les de que as en cara que y as en lis dino panto la que la que coro pera en án tila pen mo pontas conte 
------ Temperature: 0.75
nto la que la que coro pera en án tila pen mo pontas conte pano a canar panos as mentó lo cado de la de viŕiols leco poro de samo dé fues quiestel cuo ne, paro de chodo pioma no pade ad́la fuuncanpas con pias los a un coro der lá niel piel me feestar en vuma rustańas cispantiso doda yo caebile a una irca nala la ieral o vuiro colron taemanta me de estas vu ciesdo camesedes y hos cen de sé y lo lo. vurar vur abo tangror manter hóstar do.
se ul tenci
------ Temperature: 1.0
 lo lo. vurar vur abo tangror

 se la de cora de de le y y as que a de de de as de pan de se a de sende cos lar paro que me me que cales me la ros de un que de enda panto ancos sundo tan abien ter hasa as que de cando apra se le dientir de qui de puel se perenpante coro que pure dé́r hatero de calo la le sé me de que que que de lo que ra ra ada veeste hacad lo se has teledo cenda lés án la de pen tes iendo a dibentijo disiando en y de ma se que de que isto, que de que lino y de cied
------ Temperature: 0.75
ndo en y de ma se que de que isto, que de que lino y de ciedes der on esdia caler, le se vita que hana a qui, certieso en as mé timo do o terte no mal todde puica que subécela da los denta; la as rodicarolié́rnas alel ina lo mede ile meenacor y de marer la de de a de será sun ter triso tama hanos de puranta ase su de esterte ente quu manto do cale de is les as derán lancása, piro bolra en se de el de o ledar il no hos de, que lpansa paĺordo deerca n
------ Temperature: 1.0
 de el de o ledar il no hos d

...

e de de de pan que le que que de de as de de lo de de dicon de y -de ura lo que le de de de ante de en con as as a pendo pos de an ansa mir ama dí un lolnas lo dir mas que me que di que de arto de pura de rura la arir a tenen vare colel sos de an de de ale piler en de cale sir que de pien le dien a de ho pos la cor a la me se di lo no y la este momo de la que a se remo de es cien monco vento pol y demo la lo, madona mo que la core ditos cia la los le cen 
------ Temperature: 0.75
y demo la lo, madona mo que la core ditos cia la los le cen de dedo de une len de que de dulo de que ancamiser me mero, y cona se lonimáno de, su za, si man y erro saro taestoriereja que y bilabeemo me purien y de que siesta; y daste nos, arturan rugantelro en son sór si nadi casela la cuo de punecas sen atida de y que este vinas lira les puntebermirir en me de a y la inmo cana lo y deas esdo hos hen ses, mas pan cassas véas de cola véro punte lo que i
------ Temperature: 1.0
s hen ses, mas pan cassas vé

te cana a de tente se de cala ra la de me se de se as de panta de que y des de ala a de hala que de con de les lo se ante tera la a y tebo anto santos la y de y as sir los que de en rudo dies as pastén la de de que ledas que y de su la y de su tedencaca y de a tanas lo la de que lo al no o que de de lir as reno anta carlo no se de esto sos la lo que de ade as de le dé́ de que y mante la a de que dicacen tenta de bere conta qua se la pura que la que de la
------ Temperature: 0.75
 dicacen tenta de bere conta qua se la pura que la que de la vu que lo pore lilé se delenfunodel a se helenta rena li cáno atier rulo teno de dipés tepenhocua has la de dester ol se rar dar teste testerela rumas rimo paban, y de hondo que ra de o me tisdo dera ma, cerca. si ¡atena de hamó les se fercesda pacues en usod bendo sen ceńablo resentlir mas vinagon cora bieste la ros ceria dis dera aca ias abaleran qui tas pera de espon, de que na a henedas 
------ Temperature: 1.0
aca ias abaleran qui tas pera

 de de de se que de se caso de de de as de as pen la de la a que rada as de la de se varo, in de a de de reso a calele de dima ter, pira viento de mos anda mema a de caras pan lo as pen los dendo vero que se se pantel a as de me de lo ho ser ruso en en las de usa pol que te haspás mosas tes que le en de de que senas quo la de le de se con se lo de concos ni ho la anta las en de que ras a su ul pando a proso de me a pen a gera que ón lis tedas y na se ver
------ Temperature: 0.75
ndo a proso de me a pen a gera que ón lis tedas y na se ver le palgul, quigurte es acala de a en vaso que o en colro y as lo qriina paras histro que se ul pano que qui noro y sé ló solnten le alcar sol de mo cies que testaro vua de ia andos tencais de sedojos hastos entran ad hemos y hasana dórla sas ser de ara bolas racudiles le miepen le jure que en os viran derfro con esto sí ne que én que pontercelmodadalna lo adir diente de do eca coda is en le 
------ Temperature: 1.0
e pontercelmodadalna lo adir 

 se ante caso la as pure en su de de a se pon las la pen o de casá has dor dino de de que de mente de las y palen pana pure lo pia a pantaba en te de de as le cen se rante de ora la de bore de de a de an teste melerto te de as de vuebon bues la caque no y as dera, lo vóespare lo de sia penque canpo mos alsa pade o se canos coro se que ni de o su en la se vus ando de de de pia sinte que de paesta mi se, y de le didon ciestés de is anta; le any en de most
------ Temperature: 0.75
 mi se, y de le didon ciestés de is anta; le any en de mosto vensascanvoro pijor tuste mos súrte que na anse, y pi barsora as que renateman el con al la tar que, síé de cala coricenmenos las perte, palo y cora de vula do la se a as se rare en que una fues, pos es̃ámda en, qui o y su no vocoba que fuento nas bre reyo y en pen vira un de vende una ir, labro con aslma tesqua a un en que de de la dése de rolro lien de pacon gin no lacas de que ter vostci
------ Temperature: 1.0
 dése de rolro lien de pacon

de que que se puesto as lo pon a se de de de de cora te lo de que de lo tono caras lo de pen ro dendo de la de coro hala tescies se parta y moso teste lo rado pera a cade pareco con caesta dibento, de que el as la don te de adas y si en as de has, cen pasa de pemas di a di antas se fanco que su que se de a der de la ado pon pero ecad de que de has a ade paren pana se que an de peeba enta sas viera sun la sindo de de panta hodo y de de a de de mira la caro 
------ Temperature: 0.75
 sun la sindo de de panta hodo y de de a de de mira la caro da puros y canpondo ol lo la la is el se encela an tas pen mercasta de ober a y de o vorama peeca pagunas que hadas dendicenton mubia dipabes de y dero cababan rura paeste rosbérdar rado que vé́rintere me sira me fua, hoclo que dora tilo me palo pel amo que le tas sunte has lanta tascerdo la o sere no que y se cora es cuena piclas y ticas, y que se y le ditozas ave loda a que lelo y de as en as 
------ Temperature: 1.0
as, y que se y le ditozas ave

According to the web (https://www.quora.com/What-is-Temperature-in-LSTM), the temperature of a LSTM is "a hyperparameter of LSTMs (and neural networks generally) used to control the randomness of predictions by scaling the unscaled output of earlier layers before applying softmax". As seen in the Temperature notebook we were given, a value of temperature lower than 1 (as in this example) results in higher output values.

According to the web, "performing softmax on larger values makes the LSTM more confident (less input is needed to activate the output layer) but also more conservative in its samples (it is less likely to sample from unlikely candidates)". This is exactly what happens here: when we increase the temperature from 0.1 to 1, the RNN because more sensitive to the diversity in the input, resulting in an output having a lot more diversity (we can see it since we have plenty of words with more than 2-3 characters, while a temperature of 0.1 only gives really short and common words), but also resulting in more errors (the text does not really have any meaning). 

In this particular case, and even though the text obtained does not have any meaning, the temperature of 0.75 seems to give the best results: for lower values, we only get really short words and for higher values, we start to get really long words that seem to be completely made up, losing some accuracy.